In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ext = 'drive/MyDrive/CV_Images/'#concentric.mat'

In [ ]:
! pip install imagehash

     |████████████████████████████████| 812 kB 4.3 MB/s 
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295206 sha256=693cdd93ab0ba6c0b1a285e30ff122ce2b1f388abe164d6d7414b007682d637c
  Stored in directory: /root/.cache/pip/wheels/4c/d5/59/5e3e297533ddb09407769762985d134135064c6831e29a914e
Successfully built imagehash


# Load Libraries

In [ ]:
import cv2
import numpy as np
import json
import os
# import random
# import tensorflow as tf
# from keras.utils import np_utils
# from keras.models import Model, Sequential, load_model
# from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Reshape, Dropout
# from keras.preprocessing.image import ImageDataGenerator
# from keras.applications.vgg16 import VGG16
# #from keras.applications.resnet50 import ResNet50
# from keras.applications.resnet import ResNet50
# from sklearn.metrics import confusion_matrix, classification_report
# import itertools
# import pandas as pd
# import matplotlib.pyplot as plt
# from keras.callbacks import EarlyStopping
# import seaborn as sns
import shutil
from PIL import Image
import imagehash
from sklearn.cluster import AgglomerativeClustering

# Crop the folder

In [ ]:
def cropped(ele_name):
    input_path = ele_name + 'uncropped/'
    cropped_path = ele_name + 'cropped/'
    #cropped_path = 'drive/MyDrive/CV_Images/1015/cropped_train/'
    for image in os.listdir(cropped_path):
      try:
        os.remove(f'{cropped_path}{image}')
      except:
        "Meh"
    for each in os.listdir(input_path):
        ext = os.path.splitext(each)[-1]
        #print(each)
        if ext == '.json':
            filename = os.path.splitext(each)[0]
            #print(filename)
            f = open(input_path+filename+'.json')#'1019/train/IMG_6177.json')
            data = json.load(f)
            [[r1,c1],[r2,c2]] = data['shapes'][0]['points']
            img = cv2.imread(input_path+filename+'.JPG')
            if img is None:
              img = cv2.imread(input_path+filename+'.jpg')
            cropped = img[np.int64(np.floor(c1)):np.int64(np.floor(c2)),np.int64(np.floor(r1)):np.int64(np.floor(r2))]
            cv2.imwrite(cropped_path + filename +'.jpg', cropped)

In [ ]:
for elephant in os.listdir(ext):
  ele_name = ext + elephant +'/'
  cropped(ele_name)

IMG_8980
IMG_8921
IMG_8922
IMG_8957
IMG_8979
IMG_8975
IMG_8989
IMG_8972
IMG_8967
IMG_8959
IMG_8981
IMG_8973
IMG_8924
IMG_8974
IMG_8994
IMG_8978
IMG_8925
IMG_8923
IMG_8984
IMG_8960
IMG_8954
IMG_8991
IMG_8977
IMG_8953
IMG_8985
IMG_8958
IMG_8971
IMG_8976
IMG_8986
IMG_8968
IMG_8927
IMG_8990
IMG_8993
IMG_8970
IMG_8992
IMG_8916
IMG_8996
IMG_8995
IMG_8988
IMG_8987
IMG_8982
IMG_8926
IMG_8915
IMG_8966
IMG_8983
IMG_8969
IMG_9098
IMG_9040
IMG_9041
IMG_9033
IMG_9052
IMG_9046
IMG_9095
IMG_8998
IMG_8999
IMG_9014
IMG_9101
IMG_9013
IMG_9001
IMG_9211
IMG_9051
IMG_9049
IMG_9091
IMG_9009
IMG_9090
IMG_9198
IMG_9197
IMG_9195
IMG_9016
IMG_9093
IMG_9099
IMG_9096
IMG_9012
IMG_9196
IMG_9194
IMG_9048
IMG_9031
IMG_9193
IMG_9029
IMG_9002
IMG_9011
IMG_9034
IMG_9026
IMG_9036
IMG_9100
IMG_9289
IMG_8997
IMG_9094
IMG_9010
IMG_9199
IMG_9022
IMG_9050
IMG_9032
IMG_9035
IMG_9028
IMG_9038
IMG_9047
IMG_9092
IMG_9097
IMG_9015
IMG_9039
IMG_9030
IMG_9021
IMG_9037


# Imagehash + AgglomerativeClustering + test_train_validation

In [ ]:
for elephant in os.listdir(ext): #['1037','1031','1020','1012','1002']:#['1001']:#['1007','1000']:#['1039']:#'1039''1017',#['1021','1019','1015']:
  print(elephant)
  path = ext+elephant+'/cropped/'#'sim_test/'
  image_list = os.listdir(path)
  hash_matrix = np.zeros((len(image_list), len(image_list)))
  #each_count=0

  #Hashing
  for each in range(len(image_list)):
    for every in range(len(image_list)):
      hash1 = imagehash.average_hash(Image.open(path+image_list[each]))
      hash2 = imagehash.average_hash(Image.open(path+image_list[every]))
      diff = hash1-hash2
      hash_matrix[each,every] = diff
    #each_count+=1
    #print(each_count)
  print("Hashing Matrix is done")

  # Agglomerative Clustering
  cluster = AgglomerativeClustering(n_clusters=3, affinity='precomputed', linkage='average')
  pred = cluster.fit_predict(hash_matrix)
  print("Agglomerative Clustering is done")

  # Segregation
  clust0_indices, clust1_indices, clust2_indices  = [i for i in range(len(pred)) if pred[i] == 0], [i for i in range(len(pred)) if pred[i] == 1], [i for i in range(len(pred)) if pred[i] == 2]
  clust0,clust1,clust2 = [image_list[i] for i in clust0_indices], [image_list[i] for i in clust1_indices], [image_list[i] for i in clust2_indices]  
  print("Segregation is done")

  # Writing to google drive
  for each in clust0:
    shutil.copy(path+each,ext+elephant+'/c0')
    #shutil.copy(ext+'sim_test/'+each, ext+'test_sim2/c0')
  print("Cluster0 done")

  for each in clust1:
    shutil.copy(path+each,ext+elephant+'/c1')
    #shutil.copy(ext+'sim_test/'+each, ext+'test_sim2/c1')
  print("Cluster1 done")

  for each in clust2:
    shutil.copy(path+each,ext+elephant+'/c2')
    #shutil.copy(ext+'sim_test/'+each, ext+'test_sim2/c2')
  print("Cluster2 done")

1037
Hashing Matrix is done
Agglomerative Clustering is done
Segregation is done
Cluster0 done
Cluster1 done
Cluster2 done
1031
Hashing Matrix is done
Agglomerative Clustering is done
Segregation is done
Cluster0 done
Cluster1 done
Cluster2 done
1020
Hashing Matrix is done
Agglomerative Clustering is done
Segregation is done
Cluster0 done
Cluster1 done
Cluster2 done
1012
Hashing Matrix is done
Agglomerative Clustering is done
Segregation is done
Cluster0 done
Cluster1 done
Cluster2 done
1002
Hashing Matrix is done
Agglomerative Clustering is done
Segregation is done
Cluster0 done
Cluster1 done
Cluster2 done


# Combining c0,c1,c2 into cropped_test/uncropped_test and train datasets

In [ ]:
# 1040 - train(c0,c1), test(c2)
# 1015 - train(c0), test(c1,c2)
# 1004 - train(c0), test(c1,c2)
# 1019 - train(c0), test(c1,c2)
# 1021 - train(c0), test(c1,c2) clusters are very dependent on background
# 1005 - train(c0), test(c1,c2)
# 1007 - train(c1,c2), test(c0)
# 1001 - train(c0,c1), test(c2)
#['1037','1031','1020','1012','1002']
# 1031 - train(c0), test(c1,c2)
# 1012 - train(c0), test(c1,c2)
# 1002 - train(c0,c1), test(c2)
# 1020 - train(c0), test(c1,c2) -> bad dataset
# 1037 - train(c0), test(c1,c2) 
ext = 'drive/MyDrive/CV_Images/'

# Pushing the data into train, cropped_test, and uncropped_test

In [ ]:
for elephant in ['1031','1012','1020','1037']:#['1001']:#['1007']:#['1008','1018']:#['1039','1017']:#['1015','1004','1019','1021','1005']:
  #for each in os.listdir(ext+'1019/cropped_train/'):#tr_image_list:
  for each in os.listdir(ext+elephant+'/c0'):
    shutil.copy(ext+elephant+'/c0/'+each, ext+elephant+'/train/')

  for each in os.listdir(ext+elephant+'/c1'):
    shutil.copy(ext+elephant+'/c1/'+each, ext+elephant+'/cropped_test/')

  for each in os.listdir(ext+elephant+'/c2'):
    shutil.copy(ext+elephant+'/c2/'+each, ext+elephant+'/cropped_test/')

In [ ]:
for elephant in ['1002']:#['1000']:#['1008','1018']:#['1039','1017']:#['1015','1004','1019','1021','1005']:
  for each in os.listdir(ext+elephant+'/c0'):
    shutil.copy(ext+elephant+'/c0/'+each, ext+elephant+'/train/')

  for each in os.listdir(ext+elephant+'/c1'):
    shutil.copy(ext+elephant+'/c1/'+each, ext+elephant+'/train/')

  for each in os.listdir(ext+elephant+'/c2'):
    shutil.copy(ext+elephant+'/c2/'+each, ext+elephant+'/cropped_test/')

# Copying images from cropped to uncropped 

In [ ]:
for elephant in ['1037','1031','1020','1012','1002']:#['1001','1000','1007']:#['1039','1017']:#['1015','1004','1019','1021','1005','1040']:
  for each in os.listdir(ext+elephant+'/cropped_test'):
    new_each = each.split('.')[0] +'.JPG'
    try:
      shutil.copy(ext+elephant+'/uncropped/'+new_each, ext+elephant+'/uncropped_test/')
    except:
      shutil.copy(ext+elephant+'/uncropped/'+each, ext+elephant+'/uncropped_test/')
